In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv(override=True)
print("Environment variables loaded!")

Environment variables loaded!


c:\micromamba\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GOOGLE_API_KEY_3"))

models = genai.list_models()

for m in models:
    print(m.name, " | supported:", m.supported_generation_methods)


models/embedding-gecko-001  | supported: ['embedText', 'countTextTokens']
models/gemini-2.5-flash  | supported: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro  | supported: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp  | supported: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash  | supported: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001  | supported: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation  | supported: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001  | supported: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite  | supported: ['generateContent', 'countTokens', 'createCachedContent

In [5]:
def test_gemini_key(api_key: str, key_name: str) -> bool:
    """Test if a Gemini API key is valid by making a simple API call."""
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-2.0-flash')
        
        # Simple test prompt
        response = model.generate_content("Say 'Hello' in one word.")
        
        if response and response.text:
            print(f"✅ {key_name}: WORKING")
            print(f"   Response: {response.text.strip()[:50]}...")
            return True
        else:
            print(f"❌ {key_name}: No response received")
            return False
            
    except Exception as e:
        print(f"❌ {key_name}: FAILED")
        print(f"   Error Type: {type(e).__name__}")
        print(f"   Error Message: {str(e)}")
        return False

In [6]:
# Get all Google API keys from environment
google_keys = {}

# Check for various key naming patterns
key_patterns = [
    "GOOGLE_API_KEY",
    "GOOGLE_API_KEY_1",
    "GOOGLE_API_KEY_2",
    "GOOGLE_API_KEY_3",
    "GOOGLE_API_KEY_4",
    "GOOGLE_API_KEY_5",
    "GOOGLE_API_KEY_6",
    "GOOGLE_API_KEY_7",
    "GEMINI_API_KEY",
]

for key_name in key_patterns:
    value = os.getenv(key_name)
    if value:
        # Clean up the value (remove quotes if present)
        value = value.strip().strip("'\"")
        google_keys[key_name] = value

print(f"Found {len(google_keys)} API key(s):")
for key_name in google_keys:
    masked = google_keys[key_name][:10] + "..." + google_keys[key_name][-4:]
    print(f"  - {key_name}: {masked}")

Found 8 API key(s):
  - GOOGLE_API_KEY: AIzaSyBDwk...PzJk
  - GOOGLE_API_KEY_1: AIzaSyCom9...hEKo
  - GOOGLE_API_KEY_2: AIzaSyCC72...Dnag
  - GOOGLE_API_KEY_3: AIzaSyCMNi...lOvg
  - GOOGLE_API_KEY_4: AIzaSyDi97...2gDw
  - GOOGLE_API_KEY_5: AIzaSyBcpR...NJzI
  - GOOGLE_API_KEY_6: AIzaSyCzC-...RcWY
  - GOOGLE_API_KEY_7: AIzaSyBuon...1sTo


In [7]:
# Test all keys
print("=" * 60)
print("Testing API Keys")
print("=" * 60)
print()

working_keys = []
failed_keys = []

for key_name, api_key in google_keys.items():
    if test_gemini_key(api_key, key_name):
        working_keys.append(key_name)
    else:
        failed_keys.append(key_name)
    print()

# Summary
print("=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"✅ Working keys: {len(working_keys)}")
for key in working_keys:
    print(f"   - {key}")
print(f"❌ Failed keys: {len(failed_keys)}")
for key in failed_keys:
    print(f"   - {key}")

Testing API Keys

❌ GOOGLE_API_KEY: FAILED
   Error Type: ResourceExhausted
   Error Message: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 21.505468904s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free

In [ ]:
from langsmith import Client
from typing import List, Dict, Any, Optional

# Initialize LangSmith client
langsmith_client = Client()

# Configuration
LANGSMITH_PROJECT = "aws"  # Replace with your project name
THREAD_ID = "anurag-light-english-thread-20251222-060515"  # Replace with actual thread ID

# Get thread history using metadata filter (official approach)
def get_thread_history(thread_id: str, project_name: str):
    """
    Gets a history of all runs in the thread using metadata filtering.
    """
    # Filter runs by the specific thread using metadata
    # This searches for runs where metadata contains session_id, conversation_id, or thread_id
    filter_string = f'and(in(metadata_key, ["session_id","conversation_id","thread_id"]), eq(metadata_value, "{thread_id}"))'
    
    print(f"Searching for thread: {thread_id}")
    print(f"Project: {project_name}")
    print(f"Filter: {filter_string}")
    print("=" * 60)
    
    try:
        # Get all runs matching the filter
        runs = list(langsmith_client.list_runs(
            project_name=project_name, 
            filter=filter_string
        ))
        
        print(f"Total runs found: {len(runs)}")
        print()
        
        # Sort by start time (most recent first)
        runs = sorted(runs, key=lambda run: run.start_time, reverse=True)
        
        # Display all runs
        for i, run in enumerate(runs, 1):
            print(f"Run #{i}")
            print(f"  Run ID: {run.id}")
            print(f"  Name: {run.name}")
            print(f"  Run Type: {run.run_type}")
            print(f"  Status: {run.status}")
            print(f"  Start Time: {run.start_time}")
            print(f"  End Time: {run.end_time}")
            print(f"  Metadata: {run.metadata}")
            if run.error:
                print(f"  Error: {run.error}")
            if run.inputs:
                print(f"  Inputs: {str(run.inputs)[:300]}...")
            if run.outputs:
                print(f"  Outputs: {str(run.outputs)[:300]}...")
            print("-" * 60)
        
        return runs
        
    except Exception as e:
        print(f"Error retrieving thread history: {e}")
        import traceback
        traceback.print_exc()
        return []

# Alternative: Get runs using session_name (for LangGraph-style threads)
def get_thread_by_session(thread_id: str, project_name: str):
    """
    Alternative approach using session_name parameter.
    """
    print(f"\nAlternative: Using session_name parameter")
    print(f"Thread ID: {thread_id}")
    print("=" * 60)
    
    try:
        runs = list(langsmith_client.list_runs(
            project_name=project_name,
            session_name=thread_id,
            limit=100
        ))
        
        print(f"Total runs found: {len(runs)}")
        print()
        
        for i, run in enumerate(runs, 1):
            print(f"Run #{i}: {run.name} | Type: {run.run_type} | Status: {run.status}")
        
        return runs
        
    except Exception as e:
        print(f"Error: {e}")
        return []

# Execute both approaches
print("APPROACH 1: Metadata Filter (Official Documentation)")
print("=" * 60)
runs_metadata = get_thread_history(THREAD_ID, LANGSMITH_PROJECT)

print("\n" + "=" * 60)
print("APPROACH 2: Session Name")
print("=" * 60)
runs_session = get_thread_by_session(THREAD_ID, LANGSMITH_PROJECT)

# List all sessions to find available threads
print("\n" + "=" * 60)
print("List all sessions (threads) in project:")
print("=" * 60)

try:
    sessions = list(langsmith_client.list_sessions(
        project_name=LANGSMITH_PROJECT,
        limit=20
    ))
    
    print(f"Found {len(sessions)} sessions:")
    for session in sessions:
        print(f"  Session: {session.name} | Run Count: {session.run_count}")
except Exception as e:
    print(f"Error: {e}")

APPROACH 1: Metadata Filter (Official Documentation)
Searching for thread: anurag-light-english-thread-20251222-060515
Project: aws
Filter: and(in(metadata_key, ["session_id","conversation_id","thread_id"]), eq(metadata_value, "anurag-light-english-thread-20251222-060515"))
Total runs found: 46

Run #1
  Run ID: 019b44ad-39a3-7e33-9fba-d20934e2bd32
  Name: LangGraph
  Run Type: chain
  Status: error
  Start Time: 2025-12-22 06:09:21.827102
  End Time: 2025-12-22 06:09:21.973978
  Metadata: {'ls_run_depth': 0, 'thread_id': 'anurag-light-english-thread-20251222-060515'}
  Error: InvalidUpdateError("At key 'clicked_autosuggestion': Can receive only one value per step. Use an Annotated key to handle multiple values.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_CONCURRENT_GRAPH_UPDATE")Traceback (most recent call last):


  File "/usr/local/lib/python3.11/site-packages/langgraph/pregel/main.py", line 2579, in stream
    with SyncPregelLoop(


 

: 